In [1]:
# import libaries
import geopandas as gpd
import pandas as pd
import json
import numpy as np
from sqlalchemy import create_engine
import time
from datetime import date, timedelta
from rl_v2g import CarsharingEnv
import math
import stable_baselines3
from stable_baselines3 import PPO
from gym.utils.env_checker import check_env as checker_gym
from stable_baselines3.common.env_checker import check_env as checker_baselines3

# load the database credentials from the JSON file
with open('config/credentials.json') as f:
    credentials = json.load(f)

# create connection string
connection_string = f"postgresql://{credentials['username']}:{credentials['password']}@{credentials['host']}:{credentials['port']}/{credentials['database_name']}"

# create the engine with the connection string
engine = create_engine(connection_string)


C:\Users\domin\anaconda3\envs\simulation_car_sharing\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Application of Car-sharing Simulation Environment 

Choose the timespan for the simulation. The simulation will be executed chronologically, starting from the first day (2019-1-1) and continuing for subsequent days (2019-1-2, 2019-1-3, etc.). If a start date other than 2019-1-1 is selected, the "Start simulation" cell below may need to be modified.

In [2]:
# set simulation period
start_date = date(2019, 1, 1)
end_date = date(2019, 1, 3)

# calculate number of days to simulate
nr_iterations = (end_date - start_date).days

# Load data for simulation

### Car-sharing stations

In [3]:
# get station geodata, create spatial index
sql = " SELECT * FROM msc_2023_dominik.distinct_stations"
stations = gpd.read_postgis(sql, engine, geom_col='geom',crs = "EPSG:2056")
stations.sindex
stations.head()

,station_no,geom
0,2901,POINT (2555501.836 1145060.068)
1,2905,POINT (2752963.411 1260089.916)
2,2910,POINT (2501877.645 1126218.900)
3,2913,POINT (2682234.096 1243208.370)
4,2918,POINT (2736874.744 1253090.505)


### Vehicle information

In [4]:
# get vehicle data
sql = "SELECT * FROM msc_2023_dominik.vehicle_information ORDER BY vehicle_no"
vehicles = pd.read_sql(sql, engine)
vehicles.head()

,index,vehicle_category,vehicle_no,model_name,brand_name,charge_power,battery_capacity,range
0,2962,Minivan,106516,eVito 129KB Tourer Pro 3200,Mercedes-Benz,11.0,100.0,378.0
1,2963,Minivan,106517,eVito 129KB Tourer Pro 3200,Mercedes-Benz,11.0,100.0,378.0
2,2964,Minivan,106518,eVito 129KB Tourer Pro 3200,Mercedes-Benz,11.0,100.0,378.0
3,2965,Minivan,106519,eVito 129KB Tourer Pro 3200,Mercedes-Benz,11.0,100.0,378.0
4,2966,Combi,106526,Enyaq iV80,Skoda,11.0,82.0,420.0


### Reservations

In [5]:
# get daily reservations, save in dict for fast data access
delta = timedelta(days=1)
reservations_dict = {}
start_date_reservations = start_date
while start_date_reservations <= end_date:
    sql = """SELECT reservation_no, start_station_no, vehicle_no, reservationfrom_time_discrete, drive_firststart_time_discrete, 
            drive_lastend_time_discrete, reservation_duration, revenue_distance, required_soc, revenue_duration, drive_km, 
            (floor(EXTRACT(epoch FROM (date_trunc('hour', TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS')) + 
                                floor(EXTRACT(minute FROM TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS')) / 15) * interval '15 minutes' 
                                - date_trunc('hour', TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS')) - 
                                floor(EXTRACT(minute FROM TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS')) / 15) * interval '15 minutes'
                               )) / 900) * 900 + 900) / 900 AS drive_duration
            FROM msc_2023_dominik.reservations_long_time 
            WHERE  DATE(reservationfrom_discrete_date) = '{}' or  DATE(drive_firststart_discrete_date) = '{}' 
            ORDER BY reservationfrom_discrete""".format(start_date_reservations, start_date_reservations)
    reservations = pd.read_sql(sql, engine)
    reservations_dict[start_date_reservations.strftime('%Y-%m-%d')] = reservations
    start_date_reservations += delta
reservations_dict[(start_date).strftime('%Y-%m-%d')].head()

,reservation_no,start_station_no,vehicle_no,reservationfrom_time_discrete,drive_firststart_time_discrete,drive_lastend_time_discrete,reservation_duration,revenue_distance,required_soc,revenue_duration,drive_km,drive_duration
0,24519192,2886,114874,0.0,0.0,4.0,6.0,10.50,5.384615,5.25,14.0,4.0
1,24519174,1557,115969,0.0,0.0,49.0,50.0,45.75,30.500000,43.75,61.0,50.0
2,24514447,2702,114871,0.0,4.0,5.0,6.0,2.66,1.538462,7.50,4.0,2.0
3,24519221,3165,116525,0.0,0.0,96.0,96.0,0.00,0.000000,0.00,0.0,97.0
4,24519097,4407,113833,0.0,2.0,9.0,10.0,13.00,10.000000,6.25,20.0,7.0


### Electicity prices for charging

In [6]:
# get charging costs data
prices = ""
for i in range(0, 480, 5):
    price = i / 20
    prices += '"Price_chf_kwh_{}", '.format(price)

sql = """SELECT {} "Delivery day" FROM msc_2023_dominik.charging_costs WHERE "Delivery day" >=  '{}' and "Delivery day" <=  '{}' ORDER BY "Delivery day" """.format(prices, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))

charging_costs = pd.read_sql(sql, engine)
charging_costs.head()

,Price_chf_kwh_0.0,Price_chf_kwh_0.25,Price_chf_kwh_0.5,Price_chf_kwh_0.75,Price_chf_kwh_1.0,Price_chf_kwh_1.25,Price_chf_kwh_1.5,Price_chf_kwh_1.75,Price_chf_kwh_2.0,Price_chf_kwh_2.25,...,Price_chf_kwh_21.75,Price_chf_kwh_22.0,Price_chf_kwh_22.25,Price_chf_kwh_22.5,Price_chf_kwh_22.75,Price_chf_kwh_23.0,Price_chf_kwh_23.25,Price_chf_kwh_23.5,Price_chf_kwh_23.75,Delivery day
0,0.054160,0.054160,0.054160,0.054160,0.052522,0.052522,0.052522,0.052522,0.050906,0.050906,...,0.054634,0.059397,0.059397,0.059397,0.059397,0.059387,0.059387,0.059387,0.059387,2019-01-01
1,0.053848,0.053848,0.053848,0.053848,0.052436,0.052436,0.052436,0.052436,0.044785,0.044785,...,0.065669,0.064731,0.064731,0.064731,0.064731,0.063546,0.063546,0.063546,0.063546,2019-01-02
2,0.057145,0.057145,0.057145,0.057145,0.054419,0.054419,0.054419,0.054419,0.052070,0.052070,...,0.067910,0.065755,0.065755,0.065755,0.065755,0.061800,0.061800,0.061800,0.061800,2019-01-03


In [7]:
# save in dict for fast data access
delta = timedelta(days=1)
charging_costs_dict = {}
start_date_electricity = start_date
while start_date_electricity <= end_date:
    electricity_price_day = charging_costs[charging_costs["Delivery day"].dt.date == start_date_electricity].drop(["Delivery day"],axis = 1).iloc[0].values
    charging_costs_dict[start_date_electricity.strftime('%Y-%m-%d')] = electricity_price_day
    start_date_electricity += delta

### Secondary energy prices (for V2G)

In [8]:
# get v2g price data
sql = """SELECT "Timestamp", "Secondary_positive_v2g_prices_chf_kwh", "Secondary_negative_v2g_prices_chf_kwh" FROM msc_2023_dominik.v2g_prices WHERE "Timestamp" >=  '{}' and "Timestamp" <=  '{}' ORDER BY "Timestamp" """.format(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
v2g_prices = pd.read_sql(sql, engine)
v2g_prices.head()

,Timestamp,Secondary_positive_v2g_prices_chf_kwh,Secondary_negative_v2g_prices_chf_kwh
0,2019-01-01 00:00:00,0.068370,0.045588
1,2019-01-01 00:15:00,0.068838,0.045896
2,2019-01-01 00:30:00,0.068838,0.045896
3,2019-01-01 00:45:00,0.068838,0.045896
4,2019-01-01 01:00:00,0.068838,0.045896


In [9]:
# save in dict for fast data access
delta = timedelta(days=1)
v2g_price_dict = {}
start_date_v2g = start_date
while start_date_v2g <= end_date:
    v2g_price_day_positive = v2g_prices[v2g_prices['Timestamp'].dt.date == pd.Timestamp(start_date_v2g).date()].drop(["Timestamp"],axis = 1)["Secondary_positive_v2g_prices_chf_kwh"].values
    v2g_price_day_negative = v2g_prices[v2g_prices['Timestamp'].dt.date == pd.Timestamp(start_date_v2g).date()].drop(["Timestamp"],axis = 1)["Secondary_negative_v2g_prices_chf_kwh"].values
    v2g_price_dict[start_date_v2g.strftime('%Y-%m-%d')] = [v2g_price_day_positive, v2g_price_day_negative]
    start_date_v2g += delta

# Check environment

In [10]:
# check support of GPU
stable_baselines3.common.utils.get_device(device='cuda')

device(type='cuda')

In [11]:
# check if enviornment fullfils requirements of gym and stable-baselines3

# load discrete table
sql =  "SELECT * FROM discrete.discrete_weeks_{} ORDER BY vehicle_no".format(0)
data = pd.read_sql(sql, engine)
    
# load discrete planned reservation table
sql =  "SELECT * FROM msc_2023_dominik.planned_reservations_discrete_{} ORDER BY vehicle_no".format(0)
planned_reservations = pd.read_sql(sql, engine)
    
# load discrete planned reservation duration table
sql =  "SELECT * FROM msc_2023_dominik.planned_durations_discrete_{} ORDER BY vehicle_no".format(0)
planned_durations = pd.read_sql(sql, engine)
end = time.time()

# get number of vehicles
nr_vehicles = len(vehicles)

# maximal simulation length
if nr_iterations > 577:
    nr_iterations = 577
    
count = 0
# iterate over weeks (for loading weekly discrete data)
for week_nr in range(0, 1):
    # iteration for each day
    for day in range(98,99,96):
        
        # calculate number of timesteps since first day of simulation
        timesteps_since_start = count * 96
        
        # all requested days are simulated
        if count == nr_iterations:
            break
            
        # get date
        date_day = pd.to_datetime(data.columns[day-97]).date()
        date_day_string = date_day.strftime('%Y-%m-%d')
        
        # load reservations
        reservations = reservations_dict[date_day_string]
        
        # load electricity prices for charging
        electricity_price = charging_costs_dict[date_day_string]
        
        # load secondary energy prices for v2g
        v2g_price = v2g_price_dict[date_day_string]
    
        # select discrete data of day
        daily_data = data.iloc[:,day-97:day-1]
        planned_reservations_day = planned_reservations.iloc[:,day-97 + 1:day + 1] 
        planned_durations_day = planned_durations.iloc[:,day-97 + 1:day + 1] 
        
        # create environment
        env = CarsharingEnv(stations, vehicles, planned_bookings = True, v2g_penalty = 10000, penalty_per_kwh = 0, daily_data = daily_data, reservations = reservations,
                           electricity_price = electricity_price, timesteps_since_start = timesteps_since_start, v2g_price = v2g_price, planned_reservations = planned_reservations_day,
                           planned_durations = planned_durations_day)
        
        # check implementation 
        checker_gym(env)
        checker_baselines3(env)
        # count number of simulated days
        count += 1

Reset environment to timestamp:  0
Timestep:  0  Reward:  -142.7447523068973  CHF
Reset environment to timestamp:  0
Timestep:  0  Reward:  -143.59568092083157  CHF
Timestep:  1  Reward:  67.87206167943049  CHF
Timestep:  2  Reward:  133.24970879047427  CHF
Timestep:  3  Reward:  105.24589797241201  CHF
Timestep:  4  Reward:  122.77695957267933  CHF
Timestep:  5  Reward:  207.418854455931  CHF
Timestep:  6  Reward:  57.617253152360405  CHF
Timestep:  7  Reward:  76.4392001731047  CHF
Timestep:  8  Reward:  -54.62994236507963  CHF
Timestep:  9  Reward:  -87.69522586083038  CHF
Timestep:  10  Reward:  7.534486072178765  CHF
Reset environment to timestamp:  0
Timestep:  0  Reward:  -141.8036776281723  CHF
Timestep:  1  Reward:  68.41003420764895  CHF
Timestep:  2  Reward:  126.23770483976426  CHF
Timestep:  3  Reward:  101.94183996053543  CHF
Timestep:  4  Reward:  115.62012001975681  CHF
Timestep:  5  Reward:  213.33320364969478  CHF
Timestep:  6  Reward:  54.812713240268906  CHF
Timeste

# Train Agent

Start simulation by running the following cell:

In [ ]:
# get number of vehicles
nr_vehicles = len(vehicles)

# maximal simulation length
if nr_iterations > 577:
    nr_iterations = 577
    
count = 0
# iterate over weeks (for loading weekly discrete data)
for week_nr in range(0, math.ceil(nr_iterations / 7)):
    # load discrete car-sharing table
    sql =  "SELECT * FROM discrete.discrete_weeks_{} ORDER BY vehicle_no".format(week_nr)
    data = pd.read_sql(sql, engine)
    
    # load discrete planned reservation table
    sql =  "SELECT * FROM msc_2023_dominik.planned_reservations_discrete_{} ORDER BY vehicle_no".format(week_nr)
    planned_reservations = pd.read_sql(sql, engine)
    
    # load discrete planned reservation duration table
    sql =  "SELECT * FROM msc_2023_dominik.planned_durations_discrete_{} ORDER BY vehicle_no".format(week_nr)
    planned_durations = pd.read_sql(sql, engine)
        
    # iteration for each day
    for day in range(98,676,96):
        
        # calculate number of timesteps since first day of simulation
        timesteps_since_start = count * 96
        
        # measure calculation time of episode
        start = time.time()
        
        # all requested days are simulated
        if count == nr_iterations:
            break
            
        # get date
        date_day = pd.to_datetime(data.columns[day-97]).date()
        date_day_string = date_day.strftime('%Y-%m-%d')
        
        # load reservations
        reservations = reservations_dict[date_day_string]
        
        # load electricity prices for charging
        electricity_price = charging_costs_dict[date_day_string]
        
        # load secondary energy prices for v2g
        v2g_price = v2g_price_dict[date_day_string]
    
        # select discrete data of day
        daily_data = data.iloc[:,day-97:day-1]
        planned_reservations_day = planned_reservations.iloc[:,day-97 + 1:day + 1] 
        planned_durations_day = planned_durations.iloc[:,day-97 + 1:day + 1] 
        
        # reset environment at beginnning of simulation
        if count == 0:
            
            # create environment
            env = CarsharingEnv(stations, vehicles, planned_bookings = True, v2g_penalty = 10000, penalty_per_kwh = 0, daily_data = daily_data, reservations = reservations,
                           electricity_price = electricity_price, timesteps_since_start = timesteps_since_start, v2g_price = v2g_price, planned_reservations = planned_reservations_day,
                           planned_durations = planned_durations_day)
            
            # create RL model
            model = PPO("MlpPolicy",env, verbose=1, n_steps=95)
            
        else: 
            env.load_new_data(daily_data = daily_data, reservations = reservations,
                           electricity_price = electricity_price, timesteps_since_start = timesteps_since_start, v2g_price = v2g_price, planned_reservations = planned_reservations_day,
                           planned_durations = planned_durations_day)
            s = env.reset()
        # simulate day in 15 min steps
        done = False
        counter = 0
        
        # learn one episode
        model.learn(total_timesteps=95, reset_num_timesteps=False)
            
          
        end = time.time()
        print("")
        print("Total Episode Time: ", end-start)
        print("")
        count += 1

Reset environment to timestamp:  0
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\domin\anaconda3\envs\simulation_car_sharing\lib\site-packages\stable_baselines3\ppo\ppo.py:148: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 95`, after every 1 untruncated mini-batches, there will be a truncated mini-batch of size 31
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=95 and n_envs=1)
  warnings.warn(


Reset environment to timestamp:  0
Timestep:  0  Reward:  -142.52236610458573  CHF
Timestep:  1  Reward:  72.62455510384342  CHF
Timestep:  2  Reward:  133.3795585714949  CHF
Timestep:  3  Reward:  104.11928630223396  CHF
Timestep:  4  Reward:  125.28840374856962  CHF
Timestep:  5  Reward:  215.2065947442228  CHF
Timestep:  6  Reward:  56.28578238744501  CHF
Timestep:  7  Reward:  73.85166766804068  CHF
Timestep:  8  Reward:  -63.141327669622356  CHF
Timestep:  9  Reward:  -84.71748006216296  CHF
Timestep:  10  Reward:  1.4386568996248457  CHF
Timestep:  11  Reward:  -21.97728515556814  CHF
Timestep:  12  Reward:  202.30380252118348  CHF
Timestep:  13  Reward:  44.8502867978156  CHF
Timestep:  14  Reward:  44.08529778036197  CHF
Timestep:  15  Reward:  -25.156910255215568  CHF
Timestep:  16  Reward:  22.703099815567306  CHF
Timestep:  17  Reward:  48.84662324661718  CHF
Timestep:  18  Reward:  5.499006806320523  CHF
Timestep:  19  Reward:  -23.34547661236634  CHF
Timestep:  20  Reward:

In [13]:
# print network
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=17682, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=17682, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=13260, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)